In [1]:
import  numpy as np
import pandas as pd


In [2]:
data=pd.read_csv("titanic.csv")

In [5]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [7]:
columns_drop=["PassengerId","Name","Ticket","Cabin","Embarked"]

In [8]:
data_clean=data.drop(columns=columns_drop)

In [9]:
data_clean.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500
5,0,3,male,NaN,0,0,8.4583
6,0,1,male,54.0,0,0,51.8625
7,0,3,male,2.0,3,1,21.0750
8,1,3,female,27.0,0,2,11.1333
9,1,2,female,14.0,1,0,30.0708


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 66.2+ KB


In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
le=LabelEncoder()

In [15]:
data_clean["Sex"]=le.fit_transform(data_clean["Sex"])
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [14]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null int32
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


In [16]:
data_clean=data_clean.fillna(data_clean["Age"].mean())

In [17]:
data_clean.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.000000,1,0,7.2500
1,1,1,0,38.000000,1,0,71.2833
2,1,3,0,26.000000,0,0,7.9250
3,1,1,0,35.000000,1,0,53.1000
4,0,3,1,35.000000,0,0,8.0500
5,0,3,1,29.699118,0,0,8.4583
6,0,1,1,54.000000,0,0,51.8625
7,0,3,1,2.000000,3,1,21.0750
8,1,3,0,27.000000,0,2,11.1333
9,1,2,0,14.000000,1,0,30.0708


In [70]:
def entropy(col):
    counts=np.unique(col,return_counts=True)
    N=col.shape[0]
    ent=0
    for ix in counts[1]:
        p=ix/N
        ent+=(-1.0*p*np.log2(p))
    return ent

In [76]:
def info_gain(X_data,fkey,fval):
    left,right=split_data(X_data,fkey,fval)
    l=float(left.shape[0]/X_data.shape[0])
    r=float(right.shape[0]/X_data.shape[0])
    if left.shape[0]==0 or right.shape[0]==0:
        return -100000
    i_gain=entropy(X_data.Survived)-l*entropy(left.Survived)-r*entropy(right.Survived)
    return i_gain

In [84]:

def split_data(X_data,fkey,fval):
    #Binary Split
    X_Right = pd.DataFrame([],columns=X_data.columns)
    X_Left = pd.DataFrame([],columns=X_data.columns)
    
    for ix in range(X_data.shape[0]):
        val = X_data[fkey][ix]
        
        if val>fval:
            X_Right = X_Right.append(X_data.loc[ix])
        else:
            X_Left = X_Left.append(X_data.loc[ix])
            
    
    return X_Left,X_Right

In [85]:

X = data_clean[['Pclass','Sex','Age','SibSp',"Parch","Fare"]] #select these columns from dataframe
Y = data_clean[["Survived"]]

In [86]:
for fx in X.columns:
    print(fx)
    print(info_gain(data_clean,fx,data_clean[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606143
Age
0.0008836151229467681
SibSp
0.009584541813400127
Parch
0.015380754493137666
Fare
0.04214069283899541


In [134]:
class DecisionTree:
    
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        
    def train(self,X_train):
        features = ['Pclass',"Sex",'Age',"SibSp","Parch","Fare"]
        
        info_gain_list = []
        
        
        for ix in features:
            i_gain = info_gain(X_train,ix,X_train[ix].mean())
            info_gain_list.append(i_gain)
            
        selected = features[np.argmax(info_gain_list)]
        self.fkey = selected
        self.fval = X_train[selected].mean()
            
        data_left,data_right = split_data(X_train,self.fkey,self.fval)
            
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
        
        #Base Case (Leaf Node)
        if(data_left.shape[0]==0 or data_right.shape[0]==0):
            if(X_train.Survived.mean()>0.5):
                self.target = 0
            else:
                self.target = 1
            return
        #Max Depth Base Case
        if(self.depth>=self.max_depth):
            if(X_train.Survived.mean()>0.5):
                self.target = 0
            else:
                self.target = 1
            return
        
        #Recursive Case
        #Create a Left Node
        self.left = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        #Recursively Build the left Subtree
        self.left.train(data_left)
        
        self.right = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        if(X_train.Survived.mean()>0.5):
            self.target = 0
        else:
            self.target = 1
        return
    def predict(self,test):
        if test[self.fkey]>self.fval:
            
            if self.right is None:
                return self.target
            return self.right.predict(test)
        
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [135]:
split = int(0.7*(data_clean.shape[0]))

d=DecisionTree()# class DecisionTree:
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
    def train(self,xtrain):
        Features=['Pclass','Sex','Age','SibSp',"Parch","Fare"]
        i_gain=[]
        for i in Features:
            info_gains=info_gain(xtrain,i,xtrain[i].mean())
            i_gain.append(info_gains)
        self.key=Features[np.argmax(i_gain)]
        self.val=xtrain[self.key].mean()
        print(self.key)
        
        data_left,data_right=split_data(data_clean,self.key,self.val)
        data_left=data_left.reset_index(drop=True)
        data_right=data_right.reset_index(drop=True)
        
        if (data_left.shape[0]==0 or data_right.shape[0])==0:
            if xtrain.Survived.mean()>=0.5:
                self.target=0
            else :
                self.target=1
            return
        if (self.depth>=self.max_depth):
            if xtrain.Survived.mean()>=0.5:
                self.target=0
            else :
                self.target=1
            return
        self.left=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        self.right=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        if xtrain.Survived.mean()>=0.5:
            self.target=0
        else :
            self.target=1
        return

In [136]:
train_data = data_clean[:split]
test_data = data_clean[split:]

In [137]:
test_data = test_data.reset_index(drop=True)
test_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,21.0,0,0,7.8542
1,0,3,1,21.0,0,0,16.1000
2,0,1,1,61.0,0,0,32.3208
3,0,2,1,57.0,0,0,12.3500
4,1,1,0,21.0,0,0,77.9583


In [138]:

dt = DecisionTree()

In [139]:
dt.train(train_data)

In [140]:
y_pred = []

for ix in range(test_data.shape[0]):
    pred = dt.predict(test_data.loc[ix])
    y_pred.append(pred)

In [141]:
Y_pred = 1-np.array(y_pred)

In [142]:
Y_actual = np.array(test_data.Survived)

In [143]:
np.sum(Y_pred==Y_actual)/Y_pred.shape[0]

0.8134328358208955

In [144]:
###decision tree

In [145]:
from sklearn.tree import DecisionTreeClassifier

In [146]:
tree=DecisionTreeClassifier()